In [ ]:
%%bash
if [ -d "TrafficeFlowPrediction" ]; then rm -rf TrafficeFlowPrediction; fi
git clone https://github.com/newbee1905/TraficFlowPrediction -b dev/newbee1905
cd TraficFlowPrediction
pip install -r requirements.txt

In [ ]:
%%bash
python train.py --model gru

In [ ]:
%%bash
python train.py --model lstm

In [ ]:
%%bash
python train.py --model saes

In [ ]:
%%bash
python main.py